In [1]:
DEFAULT_BASE_URL = "http://pipeline:8080/%s"

In [12]:
import numpy as np
import imageio

from PIL import Image
import os

# Saving images from ndarray to images folder
# for npyfilename in filenames:
img_array = np.load("./data/X.npy")

image_path = "images"
# im = Image.fromarray(img_array[0])
# im.show()
# name = "image"
# count = 0
# for i in range(img_array.shape[0]):
#     filename = name+str(i)+".png"
#     imageio.imwrite(os.path.join(image_path,filename),img_array[i])

print("Done converting to images")

Done converting to images


In [13]:
import cloudpickle